In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

### Data Preprocessing

In [2]:
coords = read_csv("Coordinates/Coordinates.csv").values

In [3]:
img_path = "Coordinates/images"
img_dir = [os.path.join(img_path,i) for i in os.listdir(img_path)]
img_list = []
for img in img_dir:
    img = plt.imread(img)
    # Scale to [0,1]
    if (img.dtype == np.uint8):
        img = img.astype(float) / 256
    # Convert to grayscale
    img = np.mean(img,axis=-1)
    img = np.expand_dims(img,axis=2)
    img_list.append(img)
img_list = np.array(img_list)

In [4]:
img_w = 640
img_h = 360

X = img_list
Y = coords

### Simple CNN Regression Model

In [5]:
model = Sequential(
[
    layers.Conv2D(16,3,padding='same',activation='relu',input_shape=(img_h,img_w,1)),
    layers.MaxPooling2D(),
    layers.Conv2D(32,3,padding='same',activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64,3,padding='same',activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.2),
    layers.Flatten(),
    layers.Dense(128,activation='relu'),
    layers.Dense(2)
])

### 5-fold Cross Validation

In [6]:
kf = KFold(n_splits=5)

mse = list()

for train_index, test_index in kf.split(X):
    x_train, x_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    
    model.compile(loss='mse', optimizer='adam', metrics=['mean_squared_error'])
    model.fit(x_train, y_train, epochs=20, batch_size=3)
    
    results = model.predict(x_test)
    mse.append(mean_squared_error(y_test,results))

Epoch 1/20
118/118 [==============================] - 90s 766ms/step - loss: 3710.0537 - mean_squared_error: 3710.0537
Epoch 2/20
118/118 [==============================] - 85s 724ms/step - loss: 3186.0994 - mean_squared_error: 3186.0994
Epoch 3/20
118/118 [==============================] - 88s 746ms/step - loss: 2902.3079 - mean_squared_error: 2902.3076
Epoch 4/20
118/118 [==============================] - 104s 878ms/step - loss: 2312.3511 - mean_squared_error: 2312.3511
Epoch 5/20
118/118 [==============================] - 91s 767ms/step - loss: 1481.2491 - mean_squared_error: 1481.2491
Epoch 6/20
118/118 [==============================] - 88s 745ms/step - loss: 915.1205 - mean_squared_error: 915.1205
Epoch 7/20
118/118 [==============================] - 91s 771ms/step - loss: 731.6035 - mean_squared_error: 731.6035
Epoch 8/20
118/118 [==============================] - 100s 845ms/step - loss: 452.2614 - mean_squared_error: 452.2614
Epoch 9/20
118/118 [==============================] 

118/118 [==============================] - 82s 699ms/step - loss: 68.4657 - mean_squared_error: 68.4657
Epoch 12/20
118/118 [==============================] - 82s 698ms/step - loss: 69.0131 - mean_squared_error: 69.0131
Epoch 13/20
118/118 [==============================] - 82s 698ms/step - loss: 66.1951 - mean_squared_error: 66.1951
Epoch 14/20
118/118 [==============================] - 81s 691ms/step - loss: 62.9110 - mean_squared_error: 62.9110
Epoch 15/20
118/118 [==============================] - 82s 695ms/step - loss: 62.5119 - mean_squared_error: 62.5119
Epoch 16/20
118/118 [==============================] - 83s 700ms/step - loss: 47.8352 - mean_squared_error: 47.8352
Epoch 17/20
118/118 [==============================] - 81s 690ms/step - loss: 45.5161 - mean_squared_error: 45.5161
Epoch 18/20
118/118 [==============================] - 82s 697ms/step - loss: 63.1423 - mean_squared_error: 63.1423
Epoch 19/20
118/118 [==============================] - 82s 696ms/step - loss: 64.288

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 360, 640, 16)      160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 180, 320, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 180, 320, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 90, 160, 32)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 90, 160, 64)       18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 45, 80, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 45, 80, 64)        0

### Model Evaluation

In [12]:
mse

[9301.520882205858,
 1132.9324797206439,
 319.67400425667927,
 137.58038797452593,
 428.7434396408897]

#### Average Test MSE

In [14]:
np.mean(mse)

2264.0902387597193